# Continuous Bag of Words (CBOW) Model

In this exam, we will create a CBOW model using a sample Arabic traffic corpus. The corpus consists of sentences describing various traffic scenarios. The goal of the CBOW model is to predict a target word based on its surrounding context words.

In [1]:
!pip install pyarabic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 2.3 MB/s eta 0:00:00


# Importing Required Libraries

In this step, we import the necessary libraries to build and train the Continuous Bag of Words (CBOW) model.

- **TensorFlow and Keras**: Used to build the neural network model, including the layers like `Embedding`, `Dense`, and `Lambda`.
- **Tokenizer**: A utility from Keras for tokenizing and processing text data.
- **NumPy**: Used for handling numerical operations, particularly for processing arrays and data manipulation.

These libraries will provide the essential tools for text preprocessing and model development in the upcoming steps.

Add more if needed!


In [71]:
import pyarabic.araby as araby
import pyarabic.number as number
from nltk import word_tokenize
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda , Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from pyarabic.araby import (tokenize, is_arabicrange, strip_tashkeel , strip_tatweel , normalize_hamza , tokenize , sentence_tokenize ,
tokenize_with_location)
from tensorflow.keras.callbacks import EarlyStopping

# Preparing the Corpus

In this step, we initialize the corpus that will be used for training the Continuous Bag of Words (CBOW) model. The corpus consists of Arabic sentences, each of which describes different traffic scenarios.

- **Corpus**: A collection of traffic-related sentences in Arabic.

This step sets up the text data that we will use in the upcoming stages of tokenization and model training.

In [10]:
corpus = [
    "الطريق مزدحم اليوم بسبب الحادث المروري الذي حدث صباحاً ويؤدي إلى تأخير كبير في الوصول",
    "كان الطريق مزدحما للغاية والسيارات متوقفة تقريباً نتيجة الازدحام الشديد والحركة بطيئة جداً ولا تتحسن",
    "أنا أحب الذهاب إلى السوق في الصباح الباكر لتفادي الازدحام وشراء الخضروات الطازجة دون الانتظار في الطريق",
    "السيارات بطيئة بسبب الازدحام المروري في الشارع الرئيسي والتأخير في حركة المرور خلال ساعات الذروة",
    "هناك ازدحام مروري في الشارع بسبب أعمال البناء والحفريات التي تعطل حركة السيارات وتتسبب في تأخير كبير",
    "ازدحام السيارات يزداد في المساء عندما يبدأ الجميع بالعودة إلى منازلهم من العمل وتتوقف حركة المرور بالكامل",
    "الطريق السريع يشهد ازدحاما مستمرا خلال فترة الظهيرة بسبب الشاحنات الكبيرة التي تبطئ حركة السير",
    "الحافلات والسيارات عالقة في الازدحام المروري في المنطقة التجارية مما يؤدي إلى تأخير وصول الناس إلى وجهاتهم",
    "حركة المرور مزدحمة اليوم بسبب الفعاليات التي تقام في وسط المدينة مما يزيد من صعوبة الوصول إلى هناك",
    "كان من الصعب جدا القيادة على الطريق الرئيسي اليوم بسبب الازدحام الخانق الذي استمر طوال اليوم",
    "الطريق إلى المطار مزدحم اليوم بسبب الحوادث المتكررة والتأخيرات الكبيرة في حركة المرور على الطريق السريع",
    "الشارع مزدحم بالسيارات والحافلات الكبيرة مما يجعل التنقل بطيئًا جدًا ويزيد من وقت الوصول إلى العمل",
    "ازدحام السيارات في المدينة أصبح مشكلة كبيرة خاصة خلال ساعات الذروة حيث يصعب التحرك بسرعة",
    "تفاقم الازدحام في الطرق الجانبية بسبب إغلاق الطريق الرئيسي المؤدي إلى وسط المدينة لصيانة الجسر",
    "ازدحام مروري خانق يواجه السكان يوميًا خلال تنقلهم من وإلى العمل على الطرق السريعة المؤدية إلى المدينة",
    "التأخيرات المرورية اليوم ناجمة عن سوء الأحوال الجوية والضباب الذي يعيق الرؤية ويبطئ حركة السيارات",
    "حوادث السير المتكررة على الطريق الزراعي تؤدي إلى ازدحام مروري شديد وتأخير كبير في وصول السيارات",
    "كانت حركة السير اليوم غير منتظمة بسبب تنظيم حدث رياضي كبير أدى إلى إغلاق بعض الشوارع الرئيسية",
    "الأعمال الإنشائية في الشارع الرئيسي تسببت في اختناق مروري كامل وتباطؤ في حركة السيارات خلال النهار",
    "تراكم السيارات عند تقاطع الطرق الرئيسية أدى إلى ازدحام شديد وزيادة كبيرة في مدة الانتظار للوصول إلى الجهة المطلوبة"
]

#https://github.com/linuxscout/pyarabic/blob/master/doc/features.md
morphs To remove tashkeel  
and tokinize the code


In [33]:
#scurps = str([corpus for i in corpus])
#print(scurps)

[['الطريق مزدحم اليوم بسبب الحادث المروري الذي حدث صباحاً ويؤدي إلى تأخير كبير في الوصول', 'كان الطريق مزدحما للغاية والسيارات متوقفة تقريباً نتيجة الازدحام الشديد والحركة بطيئة جداً ولا تتحسن', 'أنا أحب الذهاب إلى السوق في الصباح الباكر لتفادي الازدحام وشراء الخضروات الطازجة دون الانتظار في الطريق', 'السيارات بطيئة بسبب الازدحام المروري في الشارع الرئيسي والتأخير في حركة المرور خلال ساعات الذروة', 'هناك ازدحام مروري في الشارع بسبب أعمال البناء والحفريات التي تعطل حركة السيارات وتتسبب في تأخير كبير', 'ازدحام السيارات يزداد في المساء عندما يبدأ الجميع بالعودة إلى منازلهم من العمل وتتوقف حركة المرور بالكامل', 'الطريق السريع يشهد ازدحاما مستمرا خلال فترة الظهيرة بسبب الشاحنات الكبيرة التي تبطئ حركة السير', 'الحافلات والسيارات عالقة في الازدحام المروري في المنطقة التجارية مما يؤدي إلى تأخير وصول الناس إلى وجهاتهم', 'حركة المرور مزدحمة اليوم بسبب الفعاليات التي تقام في وسط المدينة مما يزيد من صعوبة الوصول إلى هناك', 'كان من الصعب جدا القيادة على الطريق الرئيسي اليوم بسبب الازدحام الخانق الذ

In [30]:
#strip_tashkeel(scurps)

"[['الطريق مزدحم اليوم بسبب الحادث المروري الذي حدث صباحا ويؤدي إلى تأخير كبير في الوصول', 'كان الطريق مزدحما للغاية والسيارات متوقفة تقريبا نتيجة الازدحام الشديد والحركة بطيئة جدا ولا تتحسن', 'أنا أحب الذهاب إلى السوق في الصباح الباكر لتفادي الازدحام وشراء الخضروات الطازجة دون الانتظار في الطريق', 'السيارات بطيئة بسبب الازدحام المروري في الشارع الرئيسي والتأخير في حركة المرور خلال ساعات الذروة', 'هناك ازدحام مروري في الشارع بسبب أعمال البناء والحفريات التي تعطل حركة السيارات وتتسبب في تأخير كبير', 'ازدحام السيارات يزداد في المساء عندما يبدأ الجميع بالعودة إلى منازلهم من العمل وتتوقف حركة المرور بالكامل', 'الطريق السريع يشهد ازدحاما مستمرا خلال فترة الظهيرة بسبب الشاحنات الكبيرة التي تبطئ حركة السير', 'الحافلات والسيارات عالقة في الازدحام المروري في المنطقة التجارية مما يؤدي إلى تأخير وصول الناس إلى وجهاتهم', 'حركة المرور مزدحمة اليوم بسبب الفعاليات التي تقام في وسط المدينة مما يزيد من صعوبة الوصول إلى هناك', 'كان من الصعب جدا القيادة على الطريق الرئيسي اليوم بسبب الازدحام الخانق الذي 

In [35]:
#scurps =strip_tatweel(scurps)
#print(scurps)

[['الطريق مزدحم اليوم بسبب الحادث المروري الذي حدث صباحاً ويؤدي إلى تأخير كبير في الوصول', 'كان الطريق مزدحما للغاية والسيارات متوقفة تقريباً نتيجة الازدحام الشديد والحركة بطيئة جداً ولا تتحسن', 'أنا أحب الذهاب إلى السوق في الصباح الباكر لتفادي الازدحام وشراء الخضروات الطازجة دون الانتظار في الطريق', 'السيارات بطيئة بسبب الازدحام المروري في الشارع الرئيسي والتأخير في حركة المرور خلال ساعات الذروة', 'هناك ازدحام مروري في الشارع بسبب أعمال البناء والحفريات التي تعطل حركة السيارات وتتسبب في تأخير كبير', 'ازدحام السيارات يزداد في المساء عندما يبدأ الجميع بالعودة إلى منازلهم من العمل وتتوقف حركة المرور بالكامل', 'الطريق السريع يشهد ازدحاما مستمرا خلال فترة الظهيرة بسبب الشاحنات الكبيرة التي تبطئ حركة السير', 'الحافلات والسيارات عالقة في الازدحام المروري في المنطقة التجارية مما يؤدي إلى تأخير وصول الناس إلى وجهاتهم', 'حركة المرور مزدحمة اليوم بسبب الفعاليات التي تقام في وسط المدينة مما يزيد من صعوبة الوصول إلى هناك', 'كان من الصعب جدا القيادة على الطريق الرئيسي اليوم بسبب الازدحام الخانق الذ

In [36]:
#scurps = normalize_hamza(scurps)
#print(scurps)

[['الطريق مزدحم اليوم بسبب الحادث المروري الذي حدث صباحاً ويءدي ءلى تءخير كبير في الوصول', 'كان الطريق مزدحما للغاية والسيارات متوقفة تقريباً نتيجة الازدحام الشديد والحركة بطيءة جداً ولا تتحسن', 'ءنا ءحب الذهاب ءلى السوق في الصباح الباكر لتفادي الازدحام وشراء الخضروات الطازجة دون الانتظار في الطريق', 'السيارات بطيءة بسبب الازدحام المروري في الشارع الرءيسي والتءخير في حركة المرور خلال ساعات الذروة', 'هناك ازدحام مروري في الشارع بسبب ءعمال البناء والحفريات التي تعطل حركة السيارات وتتسبب في تءخير كبير', 'ازدحام السيارات يزداد في المساء عندما يبدء الجميع بالعودة ءلى منازلهم من العمل وتتوقف حركة المرور بالكامل', 'الطريق السريع يشهد ازدحاما مستمرا خلال فترة الظهيرة بسبب الشاحنات الكبيرة التي تبطء حركة السير', 'الحافلات والسيارات عالقة في الازدحام المروري في المنطقة التجارية مما يءدي ءلى تءخير وصول الناس ءلى وجهاتهم', 'حركة المرور مزدحمة اليوم بسبب الفعاليات التي تقام في وسط المدينة مما يزيد من صعوبة الوصول ءلى هناك', 'كان من الصعب جدا القيادة على الطريق الرءيسي اليوم بسبب الازدحام الخانق الذ

i used list comprehension to be used as best pracice also i remmberd how to use the the code above was commented

In [47]:
corpus = [strip_tashkeel(sentence) for sentence in corpus]
corpus = [strip_tatweel(sentence) for sentence in corpus]
corpus = [normalize_hamza(sentence) for sentence in corpus]
corpus = [araby.tokenize(sentence) for sentence in corpus]

In [48]:
print(corpus)

[['[['], ["'الطريق"], ['مزدحم'], ['اليوم'], ['بسبب'], ['الحادث'], ['المروري'], ['الذي'], ['حدث'], ['صباحا'], ['ويءدي'], ['ءلى'], ['تءخير'], ['كبير'], ['في'], ["الوصول'"], [','], ["'كان"], ['الطريق'], ['مزدحما'], ['للغاية'], ['والسيارات'], ['متوقفة'], ['تقريبا'], ['نتيجة'], ['الازدحام'], ['الشديد'], ['والحركة'], ['بطيءة'], ['جدا'], ['ولا'], ["تتحسن'"], [','], ["'ءنا"], ['ءحب'], ['الذهاب'], ['ءلى'], ['السوق'], ['في'], ['الصباح'], ['الباكر'], ['لتفادي'], ['الازدحام'], ['وشراء'], ['الخضروات'], ['الطازجة'], ['دون'], ['الانتظار'], ['في'], ["الطريق'"], [','], ["'السيارات"], ['بطيءة'], ['بسبب'], ['الازدحام'], ['المروري'], ['في'], ['الشارع'], ['الرءيسي'], ['والتءخير'], ['في'], ['حركة'], ['المرور'], ['خلال'], ['ساعات'], ["الذروة'"], [','], ["'هناك"], ['ازدحام'], ['مروري'], ['في'], ['الشارع'], ['بسبب'], ['ءعمال'], ['البناء'], ['والحفريات'], ['التي'], ['تعطل'], ['حركة'], ['السيارات'], ['وتتسبب'], ['في'], ['تءخير'], ["كبير'"], [','], ["'ازدحام"], ['السيارات'], ['يزداد'], ['في'], ['المساء'], ['عندما

# Defining Vocabulary and Model Parameters

In this step, we define key parameters that will be used to configure the CBOW model.

- **Vocabulary size**: We calculate the size of the vocabulary based on the number of unique words in the corpus. The `vocab_size` represents the total number of unique tokens (words) in the dataset plus one for padding.
  
- **Embedding size**: The `embedding_size` defines the dimensionality of the word embeddings. In this case, we set the embedding size to 10, meaning each word will be represented as a 10-dimensional vector in the embedding layer.

- **Window size**: The `window_size` defines how many words to the left and right of the target word are considered as context. Here, a window size of 2 means that two words before and two words after the target word will be used as context.

These parameters will play an essential role in shaping the CBOW model architecture.


https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer


In [54]:
tok = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tok.fit_on_texts(corpus)
sequences = tok.texts_to_sequences(corpus)
print('After converting our words in the corpus into vector of integers:')
print(sequences)

After converting our words in the corpus into vector of integers:
[[196], [16], [17], [6], [4], [48], [18], [19], [31], [49], [50], [3], [20], [21], [1], [51], [2], [32], [8], [52], [53], [33], [54], [55], [56], [7], [57], [58], [34], [22], [59], [60], [2], [61], [62], [63], [3], [64], [1], [65], [66], [67], [7], [68], [69], [70], [71], [35], [1], [72], [2], [73], [34], [4], [7], [18], [1], [23], [12], [74], [1], [5], [13], [9], [36], [75], [2], [76], [24], [14], [1], [23], [4], [77], [78], [79], [25], [80], [5], [10], [81], [1], [20], [82], [2], [26], [10], [83], [1], [84], [85], [86], [87], [88], [3], [89], [11], [37], [90], [5], [13], [91], [2], [16], [92], [93], [94], [95], [9], [96], [97], [4], [98], [27], [25], [99], [5], [100], [2], [101], [33], [102], [1], [7], [18], [1], [103], [104], [28], [105], [3], [20], [38], [106], [3], [107], [2], [108], [13], [109], [6], [4], [110], [25], [111], [1], [39], [29], [28], [112], [11], [113], [40], [3], [114], [2], [32], [11], [115], [22], 

In [55]:
data = np.concatenate(sequences)

In [56]:
vocab_size = len(tok.word_index) + 1
embedding_size = 10
window_size = 2

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/TimeseriesGenerator


In [58]:
context_length = 2 * window_size

In [59]:
generator = TimeseriesGenerator(data, data, length=context_length, batch_size=1)

# Preparing Context-Target Pairs for CBOW

In this step, we generate the context-target pairs from the tokenized sequences to train the CBOW model.

- **Context words**: For each word in a sequence, the surrounding words (within the window size) are considered as context. The context consists of the words immediately before and after the target word.
  
- **Target word**: The word in the middle of the context window is treated as the target word that the model will learn to predict.

We iterate through each sequence, collecting the context words and corresponding target words:
- For each word in a sequence, we gather the surrounding words based on the defined window size.
- The middle word is the target, and the surrounding words form the context.

Finally:
- **`X`**: An array of context words.
- **`y`**: The target words are one-hot encoded, which means they are converted into a categorical format where each word is represented as a vector of length equal to the vocabulary size.

These context-target pairs will be used to train the CBOW model to predict a target word based on its context.


In [62]:
contexts = []
targets = []

for i in range(len(generator)):
    context, target = generator[i]
    contexts.append(context[0])
    targets.append(target[0])

In [61]:
X = np.array(contexts)
y = tf.keras.utils.to_categorical(targets, num_classes=vocab_size)

# Building and Training the CBOW Model

In this step, we build and train the Continuous Bag of Words (CBOW) model using the context-target pairs created earlier.

1. **Model architecture**:
   - **Embedding layer**: This layer transforms the input context words into dense vector representations (embeddings) of size defined by `embedding_size`. The `input_dim` is set to the vocabulary size, and the `input_length` is twice the window size (since context consists of words from both sides of the target).
   
   - **Lambda layer**: This layer computes the mean of the context word embeddings. It averages the embeddings of the context words to generate a single representation that will be used to predict the target word.
   
   - **Dense layer**: This fully connected layer outputs a probability distribution over the vocabulary, using the softmax activation function. It predicts the most likely target word based on the context word embeddings.

2. **Compilation**:
   The model is compiled using the Adam optimizer and categorical cross-entropy as the loss function, which is suitable for multi-class classification tasks. Accuracy is used as a metric to evaluate the model's performance during training.

3. **Training the model**:
   The model is trained on the context-target pairs for 500 epochs. During each epoch, the model learns to predict the target word based on the context, refining its weights to improve accuracy.

4. **Saving the model weights**:
   After training, the model weights are saved to a file (`cbow_model.weights.h5`) for future use. This allows us to load the trained model later without retraining.

By the end of this step, the CBOW model will have learned to predict target words based on their surrounding context from the given corpus.

In [74]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_size),
    Dropout(0.2),
    Lambda(lambda x: tf.reduce_mean(x, axis=1)),

    Dense(units=vocab_size, activation='softmax')
])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [75]:
his = model.fit(X, y, epochs=500, verbose=1 , callbacks=[early_stopping] )
model.save_weights('cbow_model.weights.h5')

Epoch 1/500
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0614 - loss: 5.2648
Epoch 2/500
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1137 - loss: 5.0893
Epoch 3/500
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1216 - loss: 4.7118
Epoch 4/500
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1395 - loss: 4.2994
Epoch 5/500
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1642 - loss: 3.9564
Epoch 6/500
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1995 - loss: 3.6199
Epoch 7/500
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2266 - loss: 3.3563
Epoch 8/500
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2528 - loss: 3.1062
Epoch 9/500
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3019 - loss: 2.8704
Epoch 10/500
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3327 - loss: 2.6822
Epoch 11/500
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3598 - loss: 2.4937
Epoch 12/500
215/215 ━━━━━━━━━━━━━━━━━━━━

# Predicting a Word Using the Trained CBOW Model

In this step, we define a function to predict a word based on a given context using the trained CBOW model.

1. **Function: `predict_word`**:
   - **Input**: The function takes a list of context words as input. The number of context words should match the expected size (2 times the window size).
   - **Context sequence conversion**: The input context words are tokenized into a sequence of integers using the same tokenizer that was used during training.
   - **Input validation**: The function checks whether the length of the context sequence matches the expected size (2 times the window size). If not, it prints an error message.
   - **Prediction**: The tokenized context is fed into the trained CBOW model, which predicts the probability distribution over the vocabulary.
   - **Retrieve predicted word**: The predicted word is the one with the highest probability. The function retrieves the word corresponding to the predicted index from the tokenizer's word index.

2. **Example**:
   - We provide a sample context: `['الحادث', 'بسبب', 'مزدحم', 'الطريق']`.
   - The function predicts the word that fits best in this context, based on the model's learned weights.
   - The predicted word is printed along with the input context.

This function allows us to test the CBOW model by predicting words based on their surrounding context from the corpus.

In [83]:
def predict_word(context_words, model, tok, window_size=2):

    context_sequence = tok.texts_to_sequences([context_words])[0]
    expected_length = 2 * window_size
    context_sequence = np.array(context_sequence).reshape(1, -1)
    prediction = model.predict(context_sequence)
    predicted_word_index = np.argmax(prediction)
    predicted_word = tok.index_word.get(predicted_word_index, "Unknown")
    print(f"Context: {context_words}")
    print(f"Predicted word: {predicted_word}")
    return predicted_word


context_words = ['الطريق', 'بسبب', 'المروري', 'حادث']
model.load_weights('cbow_model.weights.h5')
predicted_word = predict_word(context_words, model, tok, window_size=2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Context: ['الطريق', 'بسبب', 'المروري', 'حادث']
Predicted word: المءدي
